In [1]:
#include "stdio.h"
#include "stdlib.h"
#include <iostream>
#include <vector>

/*a workaround to solve cling issue*/
#include "../macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../load_libtorch.hpp"
/*import custom defined macros*/
#include "../custom_def.hpp"
/*import libtorch header file*/
#include <torch/torch.h>

# 线性代数运算


| 函数                              | 功能                             |
| :---------------------------------: | :---------------------------------: |
| trace                             | 对角线元素之和(矩阵的迹)          |
| diag                              | 对角线元素                       |
| triu/tril                         | 矩阵的上三角/下三角，可指定偏移量 |
| mm/bmm                            | 矩阵乘法，batch的矩阵乘法         |
| addmm/addbmm/addmv/addr/baddbmm.. | 矩阵运算                         |
| t                                 | 转置                            |
| dot/cross                         | 内积/外积                        |
| inverse                           | 求逆矩阵                         |
| svd                               | 奇异值分解                       |

tensor的线性代数操作函数都集中在
`{pytorch}/aten/src/ATen/native/LinearAlgebra.cpp` 
或
`{pytorch}/aten/src/ATen/native/cuda/LinearAlgebra.cu`
两个文件中；

In [2]:
torch::Tensor x = torch::eye(5, torch::kInt64);
printT(torch::trace(x));
printT(torch::diag(x));

x = torch::randint(36,{6,6});
printT(x);
printT(torch::triu(x));
printT(torch::tril(x));

torch::trace(x) = 
5
[ CPULongType{} ]
<<--->>

torch::diag(x) = 
 1
 1
 1
 1
 1
[ CPULongType{5} ]
<<--->>

x = 
  3  22  31  31  15   0
  4   8  27   3  13  27
  3   6  24  23  25  12
 12  14   4  35   3   7
  0  24  11  12  35  33
 34  20  14  29  20  29
[ CPUFloatType{6,6} ]
<<--->>

torch::triu(x) = 
  3  22  31  31  15   0
  0   8  27   3  13  27
  0   0  24  23  25  12
  0   0   0  35   3   7
  0   0   0   0  35  33
  0   0   0   0   0  29
[ CPUFloatType{6,6} ]
<<--->>

torch::tril(x) = 
  3   0   0   0   0   0
  4   8   0   0   0   0
  3   6  24   0   0   0
 12  14   4  35   0   0
  0  24  11  12  35   0
 34  20  14  29  20  29
[ CPUFloatType{6,6} ]
<<--->>



In [3]:
torch::Tensor x = torch::randint(20,{3,4});
torch::Tensor y = torch::randint(20,{4,3});
printT(x);
printT(y);

std::cout << "/* matrix-matrix product */" << std::endl;
printT(torch::mm(x,y));

x = 
 14  12   9   4
  9  10  15  16
  0   0  15  17
[ CPUFloatType{3,4} ]
<<--->>

y = 
  6   3  13
 11  13  18
 12   4  10
 16  18  14
[ CPUFloatType{4,3} ]
<<--->>

/* matrix-matrix product */
torch::mm(x,y) = 
 388  306  544
 600  505  671
 452  366  388
[ CPUFloatType{3,3} ]
<<--->>



In [4]:
torch::Tensor a = torch::randint(20,{2,3,4});
torch::Tensor b = torch::randint(20,{2,4,3});
printT(a);
printT(b);

std::cout << "/*batch matrix-matrix product: /*" << std::endl;
std::cout << "/*If input is a (b×n×m) tensor, mat2 is a (b×m×p) tensor, /*" << std::endl;
std::cout << "/*out will be a (b×n×p) tensor.*/" << std::endl;
printT(torch::bmm(a,b));

a = 
(1,.,.) = 
  19  15   3  12
  12  15   5   1
  18   4  19   4

(2,.,.) = 
  10  18   7  10
   9   1   5  11
  10  14   5  16
[ CPUFloatType{2,3,4} ]
<<--->>

b = 
(1,.,.) = 
   7  15  18
  15  16   9
  16   2  13
  16   8   1

(2,.,.) = 
  12   2   8
  12  18   2
  12   8   3
  15  12  11
[ CPUFloatType{2,4,3} ]
<<--->>

/*batch matrix-matrix product: /*
/*If input is a (b×n×m) tensor, mat2 is a (b×m×p) tensor, /*
/*out will be a (b×n×p) tensor.*/
torch::bmm(a,b) = 
(1,.,.) = 
  598  627  528
  405  438  417
  554  404  611

(2,.,.) = 
  570  520  247
  345  208  210
  588  504  299
[ CPUFloatType{2,3,3} ]
<<--->>



In [5]:
torch::Tensor a1 = torch::randint(3, {3,4});
torch::Tensor a2 = torch::randint(3, {3,5});
torch::Tensor a3 = torch::randint(3, {5,4});
printT(a1);
printT(a2);
printT(a3);

std::cout << "/*addmm(input, mat1, mat2, *, beta=1, alpha=1)/*" << std::endl;
std::cout << "/*out = beta * input + alpha * (mat1 * mat2) */" << std::endl;
std::cout << "addmm(a1, a2, a3, 2, 3) = " << std::endl;
printT(torch::addmm(a1, a2, a3, 2, 3));

a1 = 
 0  1  2  1
 1  1  2  0
 2  1  2  2
[ CPUFloatType{3,4} ]
<<--->>

a2 = 
 0  1  2  2  1
 2  0  0  2  1
 0  2  1  0  2
[ CPUFloatType{3,5} ]
<<--->>

a3 = 
 2  1  2  1
 0  2  0  1
 0  2  2  0
 1  1  0  1
 2  1  1  0
[ CPUFloatType{5,4} ]
<<--->>

/*addmm(input, mat1, mat2, *, beta=1, alpha=1)/*
/*out = beta * input + alpha * (mat1 * mat2) */
addmm(a1, a2, a3, 2, 3) = 
torch::addmm(a1, a2, a3, 2, 3) = 
 12  29  19  11
 26  17  19  12
 16  26  16  10
[ CPUFloatType{3,4} ]
<<--->>



## 关于tensor的一些基本运算

In [4]:
torch::Tensor x = torch::ones({3,4});
x += 1;
std::cout << "x = " << std::endl << x << std::endl << std::endl;

std::cout << "x.pow(3) = " << std::endl << x.pow(3) << std::endl << std::endl;


x = 
 2  2  2  2
 2  2  2  2
 2  2  2  2
[ CPUFloatType{3,4} ]

x.pow(3) = 
 8  8  8  8
 8  8  8  8
 8  8  8  8
[ CPUFloatType{3,4} ]

